# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
from transformers import AutoTokenizer #, AutoModelForSequenceClassification

In [6]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [7]:
import importlib
# importlib.reload(util_elastic)

In [8]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [9]:
PATH_DOC =  '../data/juris_tcu_index/doc.csv'
PATH_QUERY =  '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'

In [10]:
%%time
import logging
logging.getLogger("haystack").setLevel(logging.WARNING) #WARNING, INFO

CPU times: user 204 µs, sys: 20 µs, total: 224 µs
Wall time: 227 µs


# Data load

In [11]:
df_doc = pd.read_csv(PATH_DOC)

In [12]:
df_query = pd.read_csv(PATH_QUERY)

In [13]:
df_qrel = pd.read_csv(PATH_QREL)

In [14]:
df_doc.shape, df_query.shape, df_qrel.shape

((13255, 28), (16045, 11), (94809, 3))

In [15]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE
2,21087,"A contratação de serviços por preços superiores às referências legais (Sinapi e Sicro) deve ser,...",NaN,NaN,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES
3,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",NaN,NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ
4,29370,"Em contratatações de TI, não cabe aceitar propostas de preço com incidência de encargos majorado...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ


### Quantitative calculation

Tokenization loading

In [16]:
nome_modelo_ranking_pt = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_pt = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_pt
assert os.path.exists(nome_caminho_modelo_pt), f"Path para {nome_caminho_modelo_pt} não existe!"

nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")

nome_caminho_modelo_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_3b), f"Path para {nome_caminho_modelo_3b} não existe!"

tokenizador_pt_minilm = AutoTokenizer.from_pretrained(nome_caminho_modelo_pt)
tokenizador_pt_monot5_3b = AutoTokenizer.from_pretrained(nome_caminho_modelo_3b)

/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [18]:
nome_modelo_ptt5_base = 'unicamp-dl/ptt5-base-pt-msmarco-100k-v2'
nome_caminho_modelo_ptt5_base  = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ptt5_base
assert os.path.exists(nome_caminho_modelo_ptt5_base ), f"Path para {nome_caminho_modelo_ptt5_base } não existe!"
tokenizador_ptt5_base = AutoTokenizer.from_pretrained(nome_caminho_modelo_ptt5_base)

In [19]:
def retorna_num_tokens(parm_texto:str, parm_tokenizador:AutoTokenizer):
    return len(parm_tokenizador.tokenize(parm_texto))


In [20]:
df_query['LEN_TEXT_CHAR'] = df_query['TEXT'].apply(len)
df_query['LEN_TEXT_CHAR_LOG'] = round(np.log(df_query['TEXT'].apply(len))).astype(int)
df_query['NUM_WORD'] = df_query['TEXT'].apply(lambda x: len(x.split()))
df_query['NUM_TOKENS_MONOT5_3B'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_monot5_3b)
df_query['NUM_TOKENS_MINILM'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_minilm)
df_query['NUM_TOKENS_PTT5_BASE'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_ptt5_base)


In [21]:
df_query.columns, df_qrel.columns, df_doc.columns, 

(Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
        'NORMATIVE_AUTHOR_NAME', 'LEN_TEXT_CHAR', 'LEN_TEXT_CHAR_LOG', 'NUM_WORD', 'NUM_TOKENS_MONOT5_3B', 'NUM_TOKENS_MINILM', 'NUM_TOKENS_PTT5_BASE'],
       dtype='object'),
 Index(['QUERY_ID', 'DOC_ID', 'TYPE'], dtype='object'),
 Index(['TEXT', 'ID', 'NAME', 'DATE_REFERENCE', 'CLASS', 'HAS_SCOPE_NOTE', 'HAS_DEFINITION', 'HAS_SYNONYM', 'HAS_EXAMPLE', 'HAS_ENGLISH_TRANSLATION', 'HAS_SPECIALIZATION', 'HAS_RELATED_TERM',
        'HAS_SPANISH_TRANSLATION', 'HAS_GENERALIZATION', 'TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION',
        'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION', 'COUNT_INDEX_AREA', 'COUNT_INDEX_THEME', 'COUNT_INDEX_SUBTHEME', 'COUNT_INDEX_EXTRA', 'COUNT_INDEX_TOTAL'],
       dtype='object'

In [22]:
# Verificar as colunas com valores NaN
columns_with_nan = df_query.columns[df_query.isna().any()].tolist()

print(columns_with_nan)

['REFERENCE_LIST', 'PARADIGMATIC', 'NORMATIVE_PROCESS_TYPE']


In [24]:
df_query = df_query.fillna("")

In [25]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,LEN_TEXT_CHAR,LEN_TEXT_CHAR_LOG,NUM_WORD,NUM_TOKENS_MONOT5_3B,NUM_TOKENS_MINILM,NUM_TOKENS_PTT5_BASE
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE,321,6,50,96,71,66
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE,191,5,30,54,45,48
2,21087,"A contratação de serviços por preços superiores às referências legais (Sinapi e Sicro) deve ser,...",,,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES,263,6,35,68,55,52
3,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",,,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ,188,5,27,52,39,36
4,29370,"Em contratatações de TI, não cabe aceitar propostas de preço com incidência de encargos majorado...",,,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ,230,5,36,72,53,49


In [26]:
# Verificar as colunas com valores NaN
columns_with_nan = df_doc.columns[df_doc.isna().any()].tolist()

print(columns_with_nan)

['TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION', 'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION']


In [27]:
df_doc = df_doc.fillna("")

In [28]:
# Converter as colunas para o tipo string
df_doc[columns_with_nan] = df_doc[columns_with_nan].astype(str)

Concatenate terms (by qrel) and synonyms (by doc)

In [29]:
df = df_qrel.merge(df_doc[['ID', 'NAME', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM']], how='inner', left_on='DOC_ID', right_on='ID')

In [30]:
df.shape

(94809, 9)

In [31]:
del df['ID']

In [32]:
df.head()

,QUERY_ID,DOC_ID,TYPE,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM
0,158,15147,INDEXACAO_EXTRA,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1
1,37,15147,INDEXACAO_EXTRA,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1
2,178,15147,INDEXACAO_EXTRA,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1
3,14564,15147,INDEXACAO_EXTRA,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1
4,9219,15147,INDEXACAO_EXTRA,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1


In [34]:
df['NAME_SYNONYM'] = df.apply(lambda row: row['NAME'] if row['TEXT_SYNONYM'] == '' else row['NAME'] + ' - ' + row['TEXT_SYNONYM'].rstrip('.'), axis=1)


In [35]:
df['NAME_RELATED_TERM'] = df.apply(lambda row: row['NAME'] if row['TEXT_RELATED_TERM'] == '' else row['NAME'] + ' -- ' + row['TEXT_RELATED_TERM'].rstrip('.'), axis=1).str.rstrip('.')


In [36]:
df['NAME_SYNONYM_RELATED_TERM'] = df.apply(lambda row: row['NAME_SYNONYM'] if row['TEXT_RELATED_TERM'] == '' else row['NAME_SYNONYM'] + ' -- ' + row['TEXT_RELATED_TERM'], axis=1)


In [40]:
df[(df['HAS_SYNONYM']==1) & (df['HAS_RELATED_TERM']==0)].head(1)

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,LEN_TEXT_CHAR,LEN_TEXT_CHAR_LOG,NUM_WORD,NUM_TOKENS_MONOT5_3B,NUM_TOKENS_MINILM,NUM_TOKENS_PTT5_BASE,QUERY_ID,DOC_ID,TYPE,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM,NAME_SYNONYM,NAME_RELATED_TERM,NAME_SYNONYM_RELATED_TERM
4,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE,191,5,30,54,45,48,11614,14782,SUBTEMA,Documentação,Guarda da documentação.,,1,0,Documentação - Guarda da documentação,Documentação,Documentação - Guarda da documentação


In [38]:
df = df_query.merge(df, how='inner', left_on='ID', right_on='QUERY_ID')

In [41]:
df.columns

Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
       'NORMATIVE_AUTHOR_NAME', 'LEN_TEXT_CHAR', 'LEN_TEXT_CHAR_LOG', 'NUM_WORD', 'NUM_TOKENS_MONOT5_3B', 'NUM_TOKENS_MINILM', 'NUM_TOKENS_PTT5_BASE', 'QUERY_ID', 'DOC_ID', 'TYPE', 'NAME',
       'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM', 'NAME_SYNONYM', 'NAME_RELATED_TERM', 'NAME_SYNONYM_RELATED_TERM'],
      dtype='object')

In [42]:
df.shape

(94809, 28)

In [43]:
%%time
data_index = {'term' : [],
            'synonym': [],
            'related_term': [],
            'synonym_related_term': [],}
# Ordenar o DataFrame por 'QUERY_ID' em ordem ascendente
df_sorted = df.sort_values('QUERY_ID')
# Iterar sobre os grupos
for cnt, (query_id, group) in tqdm(enumerate(df_sorted.groupby('QUERY_ID'))):
    # print(f"Query ID: {query_id}")
    # print(group)
    group_synonym = ''
    group_term = ''
    group_related_term = ''
    group_synonym_related_term = ''

    # Iterar sobre os registros do grupo
    for cnt, (_, row_doc) in enumerate(group.iterrows()):
        if cnt==0: # salva copia dos dados da query
            row_query = row_doc.copy()

        doc_id = row_doc['DOC_ID']
        # Faça o que precisa com cada registro (doc_id) para o query_id atual
        
        # Exemplo: Imprimir o doc_id
        # print(f"Doc ID: {doc_id}")

        group_term = group_term + ', ' + row_doc['NAME'] if group_term != '' else row_doc['NAME']
        #if row_doc['HAS_SYNONYM']==1:
        group_synonym = group_synonym + '; ' + row_doc['NAME_SYNONYM'] if group_synonym != '' else row_doc['NAME_SYNONYM']
        #if row_doc['HAS_RELATED_TERM']==1:
        group_related_term = group_related_term + '; ' + row_doc['NAME_RELATED_TERM'] if group_related_term != '' else row_doc['NAME_RELATED_TERM']
        # if (row_doc['HAS_RELATED_TERM']==1) or (row_doc['HAS_SYNONYM']==1):
        group_synonym_related_term = group_synonym_related_term + '; ' + row_doc['NAME_SYNONYM_RELATED_TERM'] if group_synonym_related_term != '' else row_doc['NAME_SYNONYM_RELATED_TERM']
    # Adicione aqui qualquer lógica adicional que você precise executar para cada query_id

    row_json_term= {
        'id': row_query['ID'],
        'content': row_query['TEXT'],
        'meta': {},
    }               
    for column_name in df.columns:
        if column_name not in ['TEXT', 'QUERY_ID', 'DOC_ID', 'TYPE', 'NAME', 'TEXT_SYNONYM',
                               'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM', 'NAME_SYNONYM',
                               'NAME_RELATED_TERM', 'NAME_SYNONYM_RELATED_TERM']:  # columns of doc
            row_json_term['meta'][column_name.lower()] = row_query[column_name]        
    
    row_json_synonym = row_json_term.copy()
    row_json_related_term = row_json_term.copy()
    row_json_synonym_related_term = row_json_term.copy()

    row_json_term['content'] += '\n' + group_term + '.'
    #if len(group_synonym) > 0:
    row_json_synonym['content'] += '\n' + group_synonym
    #if len(group_related_term) > 0:
    row_json_related_term['content'] += '\n' + group_related_term
    #if len(group_synonym_related_term) > 0:
    row_json_synonym_related_term['content'] += '\n' + group_synonym_related_term


    data_index['term'].append(row_json_term)
    data_index['synonym'].append(row_json_synonym)
    data_index['related_term'].append(row_json_related_term)
    data_index['synonym_related_term'].append(row_json_synonym_related_term)


16045it [00:06, 2331.59it/s]

CPU times: user 6.88 s, sys: 64.5 ms, total: 6.95 s
Wall time: 6.91 s


In [44]:
len(data_index['term'])

16045

In [45]:
list_type_index = ['term', 'synonym', 'related_term', 'synonym_related_term' ]

In [46]:
ind_test=1
for index_type in list_type_index:
    print(index_type, data_index[index_type][ind_test]['content'], '\n')


term SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, o valor do símbolo correspondente ao cargo em comissão exercido pelo funcionário, à época do seu falecimento.
Cargo em comissão, Doença, Pessoal, Viúvo, Pensão especial. 

synonym SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, o valor do símbolo correspondente ao cargo em comissão exercido pelo funcionário, à época do seu falecimento.
Cargo em comissão - Ocupante de cargo em comissão, Cargo de direção, chefia e assessoramento, Cargo comissionado, Cargo de confiança, Exercente de cargo em comissão e Cargo de direção, chefia ou assessoramento; Doença - Enfermidade e Moléstia; Pessoal; Viúvo - Viúva, Cônjuge sobrevivente, Cônjuge supérstite e Consorte supérstite; Pensão especial - Pensão especial da Lei 3738/60, Pensão especial da Lei 6782/80, Pensão da lei 3.738/60 e 

# Index creation

In [49]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu_term_exp_3_ptt5_indir_400
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_term_exp_3_ptt5_indir_400', 'uuid': '_LoconF0SimNCEuH8eA4rA', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '6623', 'store.size': '329.9mb', 'pri.store.size': '329.9mb'}

Index: indir_juris_tcu_synonym_related_term_exp_3_ptt5_indir_400
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_synonym_related_term_exp_3_ptt5_indir_400', 'uuid': 'wc4yiJ2tTnOtSlKj5cqJkQ', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '6128', 'store.size': '340.4mb', 'pri.store.size': '340.4mb'}

Index: indir_juris_tcu_term_exp_1_ptt5_indir_400
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_term_exp_1_ptt5_indir_400', 'uuid': 'o5cuc4DWQPa6i18xuvlQVw', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '6545', 'store.size': '329mb', 'pri.store.size': '329mb'}

Index: indir_juris_tcu_synonym_exp_3_ptt5_indir_400
{'health': 'ye

In [50]:
INDEX_NAME = 'indir_juris_tcu'

In [26]:
# util_elastic.delete_index('indir_juris_tcu')

index name = indir_juris_tcu_synonym_exp_user

In [52]:
%%time
dict_index = {}
for index_type in list_type_index:
    index_name_with_expansion = INDEX_NAME+'_'+index_type+'_exp_user'
    print(f"\n********Creating index {index_name_with_expansion}")
    dict_index[index_name_with_expansion] = util_elastic.create_index(parm_index_name=index_name_with_expansion,
                                                       parm_data_carga_json=data_index[index_type],
                                                       parm_embedding_dim=1024)


********Creating index indir_juris_tcu_term_exp_user

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>

********Creating index indir_juris_tcu_synonym_exp_user

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>

********Creating index indir_juris_tcu_related_term_exp_user

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de conc

In [53]:
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo
assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"


In [54]:
%%time
for index_name in dict_index:
    print(f"\n********Updating embeddings of index {index_name}")
    util_elastic.update_index_embedding_sts(parm_index=dict_index[index_name], parm_path_model=nome_caminho_modelo)


********Updating embeddings of index indir_juris_tcu_term_exp_user


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_synonym_exp_user


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_related_term_exp_user


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_synonym_related_term_exp_user


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]

CPU times: user 10min 51s, sys: 5.86 s, total: 10min 57s
Wall time: 13min 57s


In [55]:
dict_index.keys()

dict_keys(['indir_juris_tcu_term_exp_user', 'indir_juris_tcu_synonym_exp_user', 'indir_juris_tcu_related_term_exp_user', 'indir_juris_tcu_synonym_related_term_exp_user'])